# Homework 0 : verify _fivethirtyeight_ conclusions using the same dataset

We will be focusing on the following statements made from the [dataset](https://github.com/fivethirtyeight/guns-data/blob/master/interactive_data.csv) :

1. Nearly two-thirds of gun deaths are suicides.
2. More than 85 percent of suicide victims are male.
3. Around a third of all gun deaths are homicides.
4. Around two-thirds of homicide victims who are males in the age-group of 15--34 are black.
5. Women constitue only 15 percent of the total homicide victims.




## Setting up our dataset

### First we need to find a way to display tables in jupyternotebooks :

From [this](https://stackoverflow.com/a/42323522/7360943):

In [ ]:
from IPython.display import HTML, display
import tabulate
table = [["Sun",696000,1989100000],
         ["Earth",6371,5973.6],
         ["Moon",1737,73.5],
         ["Mars",3390,641.85]]
display(HTML(tabulate.tabulate(table, tablefmt='html')))


### Now we will figure out how to read csv into tables :

From [this example](https://realpython.com/python-csv/), and knowing the columns are : "Intent","Gender","Age","Race","Deaths","Population","Rate"
We use the for row loop, in order to fill an array with the row values :

In [ ]:
import csv

with open('deaths.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    deaths_table = [["Intent","Gender","Age","Race","Deaths","Population","Rate"]]
    INTENT = 1
    GENDER = 2
    AGE = 3
    RACE = 4
    DEATHS = 5
    POPULATION = 6
    RATE = 7
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        deaths_table.append([row["Intent"],row["Gender"],row["Age"],row["Race"],row["Deaths"],row["Population"],row["Rate"]])
        line_count += 1
    print(f'Processed {line_count} lines.')

Now we display (or not if it's too big a table) the results in an HTML table : 

In [ ]:
#display(HTML(tabulate.tabulate(deaths_table, tablefmt='html')))

## 1. Nearly two-thirds of gun deaths are suicides.

We will now add up all deaths, and only those that are suicides to calculate the said ratio :

In [4]:
total_suicides = 0;
for i in deaths_table:
    if deaths_table[i][INTENT] == "Suicide":
        total_suicides+=deaths_table[i][DEATHS]
print(total_suicides)

TypeError: list indices must be integers or slices, not list